In [1]:
import numpy as np
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re

In [2]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

USER_DATA = {
    'user': 'postgres',
    'password': 'admin'
}

BD_DATA = {
    'host': 'localhost',
    'database': 'tablegames',
    'port': '5432'
}

def get_psycopg2_connection(user_data, db_data):
    """
    Entrega una conexión a la base de datos presente en db_data,
    considerando el usuario en user_data

    :param user_data: Diccionario con los datos de conexión del usuario
    :param db_data: Diccionario con los datos de conexión de la base de datos
    :return: psycopg2 connection
    """
    try:
        conn = psycopg2.connect(user=user_data['user'],
                                password=user_data['password'],
                                host=db_data['host'],
                                port=db_data['port'],
                                database=db_data['database'])
        return conn
    except Exception as error:
        print("Error al conectarse a la BD \n {}".format(error))
        return None
    
def do_sql_upsert(user_data, db_data, upsert_query):
    """

    :param user_data: Diccionario con los datos de conexión del usuario
    :param db_data: Diccionario con los datos de conexión de la bd
    :param upsert_query: Upsert Query
    :return:
    """
    try:
        conn = get_psycopg2_connection(user_data, db_data)
        cur = conn.cursor()
        cur.execute(upsert_query)
        conn.commit()
        return True
    except (Exception, psycopg2.Error) as error:
        print(
            "Error en do_sql_upsert: {}".format(
                repr(error)
            )
        )
        return None
    finally:
        cur.close()
        conn.close()

def get_sql_alchemy_engine(user_data, db_data):
    """
        Retorna una engine de SqlAlchemy

    :param user_data: Diccionario con los datos del usuario
    :param db_data: Diccionario con los datos de la base de datos
    :return: Sqlalchemy Engine
    """
    try:
        engine = create_engine(
            URL(
                'postgresql+psycopg2',
                username=user_data['user'],
                password=user_data['password'],
                host=db_data['host'],
                port=db_data['port'],
                database=db_data['database']
            )
        )
        return engine
    except Exception as error:
        print("Error al generar un engine: {}".format(repr(error)))
        return None

In [3]:
GAMES_UPSERT = '''

INSERT INTO public.games(
        permalink,
        title,
        price,
        image,
        url,
        description)
    VALUES {} 
    ON CONFLICT ON CONSTRAINT games_pkey 
        DO UPDATE 
        SET price = EXCLUDED.price,
            url = EXCLUDED.url,
            description = EXCLUDED.description
'''

PRICES_INSERT = '''
INSERT INTO public.price(
        permalink,
        date,
        price)
        VALUES {}
'''

In [4]:
class DataLoader:
    def __init__(self,df):
        self.df = df
        
    def upsert_games(self,upsert_query):
        try:
            df_aux = self.df[['permalink','title','price','image','url','description']]
            values = [
                # reemplazar " por ' para casos con ' en el email
                "{}".format(value).replace(r'"', r"'")
                if value else 'NULL'
                for value in df_aux.itertuples(
                    index=False,
                    name=None)
            ]
            if do_sql_upsert(
                USER_DATA,
                BD_DATA,
                upsert_query.format(
                    ','.join(values).replace('None', 'NULL')
                )  # Es necesario reemplazar None por NULL en la consulta
            ):
                print(
                    "Juegos insertados y/o actualizados")
            else:
                raise Exception('Error en do_sql_upsert')
        except Exception as error:
            print(
                "Error en upload_data: {}".format(
                    repr(error)
                )
            )
            
    def insert_price(self,insert_query):
        try:
            self.df.date = self.df.date.astype(str)
            df_aux = self.df[['permalink','date','price']]
            
            values = [
                # reemplazar " por ' para casos con ' en el email
                "{}".format(value).replace(r'"', r"'")
                if value else 'NULL'
                for value in df_aux.itertuples(
                    index=False,
                    name=None)
            ]
            
            if do_sql_upsert(
                USER_DATA,
                BD_DATA,
                insert_query.format(
                    ','.join(values).replace('None', 'NULL')
                )):
                print(
                    "Precios insertados")
            else:
                raise Exception('Error en do_sql_upsert')
        except Exception as error:
            print('Error en insert_price')

In [40]:
class SkyshipScrapper:
    def __init__(self,base_url='https://tienda.skyship.cl/22-juegos-de-tablero'):
        self.base_url = base_url
        self.board_game_list = list()
    
    def get_next_page_url(self,page_url='https://tienda.skyship.cl/22-juegos-de-tablero'):
        try:
            if page_url == self.base_url:
                url = urllib.request.urlopen(
                    "{}".format(
                        self.base_url
                    )
                )
            else:
                url = urllib.request.urlopen(
                    "{}".format(page_url)
                )
            soup = BeautifulSoup(url,'html.parser')
            next_url = soup.find(
                "a",
                {"class":"next page-numbers"}
            ).get('href')
            return next_url
        except AttributeError as error:
            return None
        except Exception as error:
            print(
                "Error al obtener la siguiente página: {}".format(
                    repr(error)
                )
            )
            return None
    
    def get_products_url(self,page_url):
        try:
            url = urllib.request.urlopen(
                "{}".format(page_url)
            )
            soup = BeautifulSoup(url,'html.parser')
            products_url = [url.get('href') for url in soup.find_all(href=re.compile("/juegos-de-tablero/"))]
            return list(set(products_url))
        except Exception as error:
            print(
                "Error en get_products_url: {}".format(
                    repr(error)
                )
            )
            return None
    
    def get_product_info(self,page_url):
        try:
            url = urllib.request.urlopen(
                '{}'.format(
                    page_url
                )
            )
            soup = BeautifulSoup(url,'html.parser')
            
            #delete strong tags
            for tag in soup.find_all('strong'):
                tag.replaceWith('')
            for tag in soup.find_all('b'):
                tag.replaceWith('')
            for tag in soup.find_all('i'):
                tag.replaceWith('')
            for tag in soup.find_all('em'):
                tag.replaceWith('')
                
            product_dict = dict()
            
            #get image
            image_div = soup.find('div',{'class':'easyzoom easyzoom-product'})
            product_dict['image'] = image_div.a.get('href')
            
            #get title and price
            summary_div = soup.find('div',{'class':'product_header_container clearfix'})
            product_dict['title'] = summary_div.h1.contents[0].contents[0].replace('(','').replace(')','')
            product_dict['price'] = summary_div.findAll('span',{'class':'product-price'})[0].contents[0]
            
            #get description
            description_div = soup.find(
                'div',
                {'class':'product-description'}
            ).find(
                'div',
                {'class':'rte-content'}
            )
            

            if description_div != None:
                for tag in description_div.find_all('span'):
                    tag.replaceWith('')
                for tag in description_div.find_all('a'):
                    tag.replaceWith('')
                for tag in description_div.find_all('iframe'):
                    tag.replaceWith('')
                    
                description_p = [
                    description.contents[0] 
                    for description in description_div.findAll('p') 
                ] 


                product_dict['description'] = '\n '.join(description_p)
            else:
                product_dict['description'] = 'Descripción no proporcionada.'
            product_dict['url'] = page_url
            
            self.board_game_list.append(product_dict)
        except Exception as error:
            print(page_url)
            print(error)
            
    def recursive_scrapper(self,page_url):
        urls = self.get_products_url(page_url)
        for product_url in urls:
            self.get_product_info(product_url)
        next_url = self.get_next_page_url(page_url)
        if next_url:
            print("Next Page: {}".format(next_url))
            return self.recursive_scrapper(next_url)
        else:
            return
        
    def get_contents(self):
        try:
            self.recursive_scrapper(self.base_url)
            df = pd.DataFrame(self.board_game_list)
            df['date'] = datetime.datetime.today()
            return df
        except Exception as error:
            print(
                "Error en get_context: {}".format(
                    repr(error)
                )
            )
            return None
        
        

In [41]:
skyship_scrapper = SkyshipScrapper()

In [42]:
df_skyship = skyship_scrapper.get_contents()

In [43]:
df_skyship.head()

,description,image,price,title,url,date
0,"Bienvenido a casa, buen Cazador. La sangre de ...",https://tienda.skyship.cl/412-thickbox_default...,$ 32.990,Bloodborne: El juego de cartas,https://tienda.skyship.cl/juegos-de-tablero/84...,2020-03-08 16:27:46.022247
1,"El Castillo Dragón, el centro de poder más ant...",https://tienda.skyship.cl/494-thickbox_default...,$ 41.990,Dragon Castle,https://tienda.skyship.cl/juegos-de-tablero/92...,2020-03-08 16:27:46.022247
2,"Un anillo para el Señor Oscuro, sobre el trono...",https://tienda.skyship.cl/521-thickbox_default...,$ 26.990,El Señor de los Anillos: El Enfrentamiento,https://tienda.skyship.cl/juegos-de-tablero/95...,2020-03-08 16:27:46.022247
3,Todo el mundo tiene secretos.,https://tienda.skyship.cl/319-thickbox_default...,$ 12.990,3 Secretos,https://tienda.skyship.cl/juegos-de-tablero/64...,2020-03-08 16:27:46.022247
4,Es una época de guerras y contiendas en el Jap...,https://tienda.skyship.cl/321-thickbox_default...,$ 9.990,Age of War,https://tienda.skyship.cl/juegos-de-tablero/64...,2020-03-08 16:27:46.022247


In [55]:
class SkyshipTransformer:
    def __init__(self,df):
        self.df = df
    def create_permalink(self):
        self.df['permalink'] = 'skyship-'+ self.df.title.str.replace(' ','-')
    def clean_date(self):
        self.df.date = self.df.date.dt.date
    def clean_price(self):
        self.df.price = self.df.price.str.replace(u'\xa0', u' ')
        self.df.price = self.df.price.str.replace(r'\$','').replace('.','')
        self.df.price = self.df.price.str.replace('.','')
        self.df.price = self.df.price.astype(int)
    def fill_description(self):
        self.df.description.fillna('SIN DESCRIPCIÓN',inplace=True)
    def drop_duplicates(self):
        self.df.drop_duplicates('permalink',keep='first',inplace=True)
    
    def transform_data(self):
        self.create_permalink()
        self.clean_date()
        self.clean_price()
        self.fill_description()
        self.drop_duplicates()
        return self.df

In [56]:
skyship_transformer = SkyshipTransformer(df_skyship.copy())
df_skyship_clean = skyship_transformer.transform_data()

In [57]:
df_skyship_clean.head()

,description,image,price,title,url,date,permalink
0,"Bienvenido a casa, buen Cazador. La sangre de ...",https://tienda.skyship.cl/412-thickbox_default...,32990,Bloodborne: El juego de cartas,https://tienda.skyship.cl/juegos-de-tablero/84...,2020-03-08,skyship-Bloodborne:-El-juego-de-cartas
1,"El Castillo Dragón, el centro de poder más ant...",https://tienda.skyship.cl/494-thickbox_default...,41990,Dragon Castle,https://tienda.skyship.cl/juegos-de-tablero/92...,2020-03-08,skyship-Dragon-Castle
2,"Un anillo para el Señor Oscuro, sobre el trono...",https://tienda.skyship.cl/521-thickbox_default...,26990,El Señor de los Anillos: El Enfrentamiento,https://tienda.skyship.cl/juegos-de-tablero/95...,2020-03-08,skyship-El-Señor-de-los-Anillos:-El-Enfrentami...
3,Todo el mundo tiene secretos.,https://tienda.skyship.cl/319-thickbox_default...,12990,3 Secretos,https://tienda.skyship.cl/juegos-de-tablero/64...,2020-03-08,skyship-3-Secretos
4,Es una época de guerras y contiendas en el Jap...,https://tienda.skyship.cl/321-thickbox_default...,9990,Age of War,https://tienda.skyship.cl/juegos-de-tablero/64...,2020-03-08,skyship-Age-of-War


In [58]:
skyship_dataloader = DataLoader(df_skyship_clean)
skyship_dataloader.upsert_games(GAMES_UPSERT)
skyship_dataloader.insert_price(PRICES_INSERT)

Juegos insertados y/o actualizados
Precios insertados
